In [ ]:
import json
import os
import re
from pathlib import Path

import pandas as pd

In [ ]:
os.chdir('..')

In [ ]:
root_folder = Path("path/to/data")
if not root_folder.exists():
    raise Exception("Path does not exist: {}".format(root_folder))

# get all folders in root_folder
folders = root_folder.iterdir()


# filter using exps
def filter_name(p: Path):
    return p.is_dir()


folders = sorted([f for f in folders if filter_name(f)])
pattern = re.compile(r"^([^|]+).*")
# pattern = re.compile(r"^([^_]+).*")
lines: dict[Path, str] = {path: pattern.match(path.name).group(1) for path in folders}
print(lines)

In [ ]:
data = []
for path, name in lines.items():
    assert path.is_dir(), path
    file_path = path / "result.json"
    if not os.path.isfile(file_path):
        print(f"{file_path} not exists")
        continue
    with open(file_path) as f:
        rst = json.load(f)
    line = {
        "name": name[: -len(str(rst["max_length"])) -1],
        "length": rst["max_length"],
        "ppl": rst["ppl"],
    }
    data.append(line)
df = pd.DataFrame(data)
print(df)

In [ ]:
pivot = df.pivot(index="name", columns="length", values="ppl")
print(pivot)

In [ ]:
with open(str(root_folder) + ".md", "w") as f:
    f.write(pivot.round(2).to_markdown())